In [6]:
using Distributions
using Optim

In [15]:
function generate_obs(n, params; rho = -2, verbose=false)
    
    delta = params[1:n]
    beta = params[n+1:2*n]    
    lambda = exp(rho)/(1+exp(rho))
    
    x= rand(Normal(0.0, 1.0),n)
    e = rand(Logistic(),n)
    
    a = []
    a_next = []
    for i=1:n
        append!(a_next,[[]])
        append!(a,[[0,1]])
    end
    eq = false
    
    u_lower = zeros(n)
    u_upper = zeros(n)
    
    print(a, " ", a_next)
    #iterative best responses
    while !eq
        eq = true
        #compute upper and lower bounds on utility...
        u_lower = beta.*x .+ e
        u_upper = u_lower[:]
        
        #compute correct bounds for each player
        for i=1:n
            for j=1:(n-1)
                j_index = mod(i+j-1,n)+1
                u_upper[i] = u_upper[i] + delta[i]*maximum(a[j_index])
                u_lower[i] = u_lower[i] + delta[i]*minimum(a[j_index])
            end
            
            #check to see if 0 or 1 is undominated
            if u_lower[i] <=0
                a_next[i] = append!(a_next[i],0)
            end
                
            if u_upper[i] >=0
                a_next[i] = append!(a_next[i],1)
            end
            if verbose
                println(i," ", a[i]," ", a_next[i]," ", (a_next[i] == a[i]),  " ", eq)
            end
            eq = eq & (a_next[i] == a[i])
        end
        if verbose
            println("----")
        end
        a =  a_next
        a_next = []
        for i=1:n
            append!(a_next,[[]])
        end
    end
    
    #reshape a correctly/select equilibria if mult
    a_return = zeros(n)
    select = rand(Categorical([lambda, (1-lambda)]))
    
    for i=1:n
        a_return[i] = a[i][1]
        #multiple eq
        if length(a[i])>1
            if verbose
                println(u_upper,u_lower)
                println(a)
            end
            a_return[i] = a[i][select]
            if verbose
                println(select," ", a_return)
                println("----")
            end
        end
        
    end
    
    return a_return,x
end


function gen_data(N;n=3,params= [.6, .7, .8, -3,-3.5,-4])
    Y = zeros(n,N)
    X = zeros(n,N)
    for i=1:N
        y,x = generate_obs(n,params)
        Y[:,i] = y
        X[:,i] = x
    end
    return Y,X
end

gen_data(1;n=2,params=[])

[0.6, 0.7, 0.8][-3.0, -3.5, -4.0]
Any[[0, 1], [0, 1], [0, 1]] Any[Any[], Any[], Any[]]

([1.0; 1.0; 0.0], [0.160328; -0.419196; 0.962927])

Any[Any[], Any[], Any[]]
